In [11]:
%matplotlib inline
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
from scipy.stats import poisson, kurtosis
import matplotlib.patches as mpatches
from functools import partial
import pickle
import os
import demixing as dm
from demixing import MLP, HiddenLayer

nneuron = 61
min_angle = -90
max_angle = 90
sprefs = np.linspace(min_angle, max_angle, nneuron)
eps = np.finfo(np.float64).eps
sigtc_sq = float(10**2)

In [2]:
pkl_file = open('readout.pkl', 'rb')
posts, testsets = pickle.load(pkl_file)

In [7]:
nns = {}
for i in range(1, 401):
    file_name = 'Minicon_Outputs/output_nn_tests_2/nn_tests_2_' + str(i) + '.pkl'
    if os.path.isfile(file_name):
        pkl_file = open(file_name, 'rb')
        nn, nnx, valid_mse, _, _ = pickle.load(pkl_file)
        nns[i] = (nn, nnx, valid_mse)

In [12]:
for i in range(1, len(nns) + 1):
    print i
    nn, nnx, valid_mse = nns[i]
    x = []
    y = []
    for s_i in range(31):
        x.append(dm.get_hu_responses(testsets[s_i], nn))
        y.append(np.array((1/posts[s_i]['var_s1'], 1/posts[s_i]['var_s2'])))
    y = np.concatenate(y, axis=1).T
    x = np.concatenate(x)
    inds = range(len(x))
    np.random.shuffle(inds)
    x_shuf = x[inds]
    y_shuf = y[inds]
    validset = x_shuf[0:2000], y_shuf[0:2000]
    trainset = x_shuf[2000:], y_shuf[2000:]
    
    hus, vpost = validset
    weights = np.linalg.lstsq(trainset[0], trainset[1])[0]
    lin_preds = np.dot(hus, weights)
    sum_preds = np.sum(hus, axis=1)
    kurt_preds = kurtosis(hus, axis=1)
    vp = np.concatenate((vpost.T[0], vpost.T[1]))
    lin_preds = np.concatenate((lin_preds.T[0], lin_preds.T[1]))
    sum_preds = np.concatenate((sum_preds, sum_preds))
    kurt_preds = np.concatenate((kurt_preds, kurt_preds))

    lin_corr = np.corrcoef(vp, lin_preds)
    sum_corr = np.corrcoef(vp, sum_preds)
    kurt_corr = np.corrcoef(vp, kurt_preds)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


KeyboardInterrupt: 